In [1]:
!pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.5/105.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 36.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 60.0 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117031 sha256=033cc856210eae244ee698709cd6294d24257dfe37d69de244cff79a932c9860
  Stored in directory: /root/.cache/pip/wheels/d6/6d/5d/5b73fa0f46d01a793713f8859201361e9e581ced8c75e5c6a3
Successfully built fire
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the

In [2]:
from PIL import Image
import datasets
import cv2
import numpy as np
from tqdm.auto import tqdm
from deepface.modules import modeling, preprocessing
from deepface.extendedmodels import Race, Gender
import torch
from torchvision import transforms
import numpy as np
import cv2
from PIL import Image
import tensorflow as tf
import os
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import precision_recall_fscore_support

2024-07-24 18:33:49.021401: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-24 18:33:49.021537: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-24 18:33:49.155943: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
if not os.path.exists('/root/.deepface/weights'):
    os.mkdir('/root/.deepface')
    os.mkdir('/root/.deepface/weights')
race_model = modeling.build_model("Race").model
gender_model = modeling.build_model("Gender").model

24-07-24 18:34:04 - race_model_single_batch.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/race_model_single_batch.h5
To: /root/.deepface/weights/race_model_single_batch.h5
100%|██████████| 537M/537M [00:02<00:00, 199MB/s]  


24-07-24 18:34:09 - gender_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/gender_model_weights.h5
To: /root/.deepface/weights/gender_model_weights.h5
100%|██████████| 537M/537M [00:01<00:00, 382MB/s] 


In [4]:
def resize_and_combine_classes(sample):
    
    # Combine East and SouthEast asian
    if sample["race"] in {0,6}:
        sample["race"] = 0
    # Preprocessing
    img_content = np.array(sample["image"])
    # Convert RGB image to BGR
    img_content = cv2.cvtColor(img_content, cv2.COLOR_RGB2BGR)

    # resize input image
    img_content = preprocessing.resize_image(img=img_content, target_size=(224, 224))
    sample["image"] = np.array(img_content)
    return sample

In [5]:
# Load training data
train_ds = datasets.load_dataset('HuggingFaceM4/FairFace', '1.25', split='train', verification_mode="no_checks")

Generating train split:   0%|          | 0/86744 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10954 [00:00<?, ? examples/s]

In [6]:

class ResizeAndCombineClasses(object):
    def __init__(self, size):
        pass
#         self.size = size  # Set the target size for resizing
#         self.resize = transforms.Resize(self.size)  # Initialize the resize transformation

    def __call__(self, sample):
        image, race, gender = sample['image'], sample['race'], sample['gender']

        # Combine East and SouthEast Asian classes
        if race in {0, 6}:
            race = 0

        # Preprocessing
        image = np.array(image)
        # Convert RGB image to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # resize input image
        image = preprocessing.resize_image(img=image, target_size=(224, 224))

#         # Convert to tensor (optional here depending on subsequent use)
#         image = transforms.ToTensor()(image)

        return {'image': np.array(image).squeeze(), 'race': race, 'gender': gender}


In [7]:


class HuggingFaceTorchDataset(Dataset):
    def __init__(self, hf_dataset):
        self.hf_dataset = hf_dataset
        self.transform = ResizeAndCombineClasses(size=(224, 224))

    def __len__(self):
        return len(self.hf_dataset)

    def __getitem__(self, idx):
        sample = self.hf_dataset[idx]
        sample = self.transform(sample)
        image = sample['image']
        race = sample['race']
        gender = sample['gender']
        return image, race, gender

# Load training data
train_hf_dataset = load_dataset('HuggingFaceM4/FairFace', '1.25', split='train', verification_mode="no_checks")
train_dataset = HuggingFaceTorchDataset(train_hf_dataset)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

In [8]:
ref = {
        "Man": 0,
        "Woman": 1,
        "asian": 0, 
        "indian": 1, 
        "black": 2, 
        "white": 3, 
        "middle eastern": 4, 
        "latino hispanic": 5, 
    }

In [9]:
correct_races = correct_genders = total = 0
all_race_preds = []
all_gender_preds = []
true_race_preds = []
true_gender_preds = []
for images, races, genders in tqdm(train_loader):
#     import pdb; pdb.set_trace()
    race_preds = race_model(np.array(images).squeeze())
    race_preds = np.argmax(race_preds, axis=1)
    race_preds = [ref[Race.labels[pred]] for pred in race_preds]
    correct_races += np.sum(race_preds == np.array(races))
    all_race_preds += race_preds
    true_race_preds += races
    total += len(races)
    
    gender_preds = gender_model(np.array(images).squeeze())
    gender_preds = np.argmax(gender_preds, axis=1)
    gender_preds = [ref[Gender.labels[pred]] for pred in gender_preds]
    correct_genders += np.sum(gender_preds == np.array(genders))
    all_gender_preds += gender_preds
    true_gender_preds += genders

  0%|          | 0/10843 [00:00<?, ?it/s]

In [10]:
train_acc_race = (correct_races / total)*100
train_acc_gender = (correct_genders / total)*100
print(f"Race prediction accuracy on training data: {train_acc_race:.2f}%")
print(f"Gender prediction accuracy on training data: {train_acc_gender:.2f}%")

Race prediction accuracy on training data: 37.70%
Gender prediction accuracy on training data: 74.32%


In [11]:
# Calculate F-score, precision, recall - Race - Training set 

precision, recall, f_score_weighted, _ = precision_recall_fscore_support(true_race_preds, all_race_preds, average='weighted')
_, _, f_score_macro, _ = precision_recall_fscore_support(true_race_preds, all_race_preds, average='macro')
_, _, f_score_micro, _ = precision_recall_fscore_support(true_race_preds, all_race_preds, average='micro')
train_acc = np.sum(np.array(all_race_preds) == np.array(true_race_preds)) / total
print(f"Training set metrics - Race (Deepface) \n" + "="*40)
print(f"Accuracy: {train_acc:.4f} Precision: {precision:.4f}, Recall: {recall:.4f}, F-Score(Weighted): {f_score_weighted:.4f}, F-Score(Micro): {f_score_micro:.4f}, F-Score(Macro): {f_score_macro:.4f}")

Training set metrics - Race (Deepface) 
Accuracy: 0.3770 Precision: 0.3972, Recall: 0.3770, F-Score(Weighted): 0.3412, F-Score(Micro): 0.3770, F-Score(Macro): 0.3124


In [12]:
# Calculate F-score, precision, recall - Gender - Training set 

precision, recall, f_score_weighted, _ = precision_recall_fscore_support(true_gender_preds, all_gender_preds, average='weighted')
_, _, f_score_macro, _ = precision_recall_fscore_support(true_gender_preds, all_gender_preds, average='macro')
_, _, f_score_micro, _ = precision_recall_fscore_support(true_gender_preds, all_gender_preds, average='micro')
train_acc = np.sum(np.array(all_gender_preds) == np.array(true_gender_preds)) / total
print(f"Training set metrics - Gender (Deepface) \n" + "="*40)
print(f"Accuracy: {train_acc:.4f} Precision: {precision:.4f}, Recall: {recall:.4f}, F-Score(Weighted): {f_score_weighted:.4f}, F-Score(Micro): {f_score_micro:.4f}, F-Score(Macro): {f_score_macro:.4f}")

Training set metrics - Gender (Deepface) 
Accuracy: 0.7432 Precision: 0.7624, Recall: 0.7432, F-Score(Weighted): 0.7349, F-Score(Micro): 0.7432, F-Score(Macro): 0.7315


In [13]:
# Load validation data as test data
valid_ds = datasets.load_dataset('HuggingFaceM4/FairFace', '1.25', split="validation", verification_mode="no_checks")
# valid_ds = valid_ds.shuffle(seed=42).select([i for i in range(6_000)]) # Take only first 6_000 images
valid_dataset = HuggingFaceTorchDataset(valid_ds)

# DataLoader
valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=True)

In [14]:
correct_races = correct_genders = total = 0
all_race_preds = []
all_gender_preds = []
true_race_preds = []
true_gender_preds = []
for images, races, genders in tqdm(valid_loader):
#     import pdb; pdb.set_trace()
    race_preds = race_model(np.array(images).squeeze())
    race_preds = np.argmax(race_preds, axis=1)
    race_preds = [ref[Race.labels[pred]] for pred in race_preds]
    correct_races += np.sum(race_preds == np.array(races))
    all_race_preds += race_preds
    true_race_preds += races
    total += len(races)
    
    gender_preds = gender_model(np.array(images).squeeze())
    gender_preds = np.argmax(gender_preds, axis=1)
    gender_preds = [ref[Gender.labels[pred]] for pred in gender_preds]
    correct_genders += np.sum(gender_preds == np.array(genders))
    all_gender_preds += gender_preds
    true_gender_preds += genders

  0%|          | 0/1370 [00:00<?, ?it/s]

In [15]:
valid_acc_race = (correct_races / total)*100
valid_acc_gender = (correct_genders / total)*100
print(f"Race prediction accuracy on validation data: {valid_acc_race:.2f}%")
print(f"Gender prediction accuracy on validation data: {valid_acc_gender:.2f}%")

Race prediction accuracy on validation data: 37.37%
Gender prediction accuracy on validation data: 74.08%


In [16]:
# Calculate F-score, precision, recall - Race - Validation set 

precision, recall, f_score_weighted, _ = precision_recall_fscore_support(true_race_preds, all_race_preds, average='weighted')
_, _, f_score_macro, _ = precision_recall_fscore_support(true_race_preds, all_race_preds, average='macro')
_, _, f_score_micro, _ = precision_recall_fscore_support(true_race_preds, all_race_preds, average='micro')
train_acc = np.sum(np.array(all_race_preds) == np.array(true_race_preds)) / total
print(f"Validation set metrics - Race (Deepface) \n" + "="*40)
print(f"Accuracy: {train_acc:.4f} Precision: {precision:.4f}, Recall: {recall:.4f}, F-Score(Weighted): {f_score_weighted:.4f}, F-Score(Micro): {f_score_micro:.4f}, F-Score(Macro): {f_score_macro:.4f}")

Validation set metrics - Race (Deepface) 
Accuracy: 0.3737 Precision: 0.3970, Recall: 0.3737, F-Score(Weighted): 0.3404, F-Score(Micro): 0.3737, F-Score(Macro): 0.3102


In [17]:
# Calculate F-score, precision, recall - Gender - Validation set 

precision, recall, f_score_weighted, _ = precision_recall_fscore_support(true_gender_preds, all_gender_preds, average='weighted')
_, _, f_score_macro, _ = precision_recall_fscore_support(true_gender_preds, all_gender_preds, average='macro')
_, _, f_score_micro, _ = precision_recall_fscore_support(true_gender_preds, all_gender_preds, average='micro')
train_acc = np.sum(np.array(all_gender_preds) == np.array(true_gender_preds)) / total
print(f"Validation set metrics - Gender (Deepface) \n" + "="*40)
print(f"Accuracy: {train_acc:.4f} Precision: {precision:.4f}, Recall: {recall:.4f}, F-Score(Weighted): {f_score_weighted:.4f}, F-Score(Micro): {f_score_micro:.4f}, F-Score(Macro): {f_score_macro:.4f}")

Validation set metrics - Gender (Deepface) 
Accuracy: 0.7408 Precision: 0.7590, Recall: 0.7408, F-Score(Weighted): 0.7329, F-Score(Micro): 0.7408, F-Score(Macro): 0.7297
